In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
import random
import numpy as np
import networkx as nx

In [ ]:
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [ ]:
G = nx.Graph()
edge_list = data.edge_index.t().tolist()
G.add_edges_from(edge_list)


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00


In [ ]:
lid = {node: len(list(G.neighbors(node))) for node in G.nodes()}  # Degree as LID
hub_scores, auth_scores = nx.hits(G, max_iter=100, normalized=True)


In [ ]:
lid_tensor = torch.tensor([lid[i] for i in range(data.num_nodes)], dtype=torch.float32).unsqueeze(1)
hub_tensor = torch.tensor([hub_scores[i] for i in range(data.num_nodes)], dtype=torch.float32).unsqueeze(1)
auth_tensor = torch.tensor([auth_scores[i] for i in range(data.num_nodes)], dtype=torch.float32).unsqueeze(1)

In [ ]:
data.x = torch.cat([data.x, lid_tensor, hub_tensor, auth_tensor], dim=1)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.gat1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        self.gat2 = GATConv(8 * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.gat1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.gat2(x, edge_index)
        return x

In [ ]:
model = GAT(data.num_node_features, 64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
embeddings = model(data.x, data.edge_index).detach().cpu().numpy()

In [ ]:
positive_edges = data.edge_index.t().cpu().numpy()
positive_edges = [tuple(e) for e in positive_edges if e[0] < e[1]]


In [ ]:
num_samples = len(positive_edges)
all_nodes = list(range(data.num_nodes))
negative_edges = set()
while len(negative_edges) < num_samples:
    u, v = random.sample(all_nodes, 2)
    if (u, v) not in positive_edges and (v, u) not in positive_edges:
        negative_edges.add((u, v))


In [ ]:
X, y = [], []
for u, v in positive_edges[:num_samples]:
    X.append(embeddings[u] * embeddings[v])  # Hadamard product
    y.append(1)
for u, v in list(negative_edges):
    X.append(embeddings[u] * embeddings[v])
    y.append(0)

X = np.array(X)
y = np.array(y)

In [ ]:
indices = np.arange(len(X))
np.random.shuffle(indices)
split = int(0.8 * len(X))
X_train, X_test = X[indices[:split]], X[indices[split:]]
y_train, y_test = y[indices[:split]], y[indices[split:]]

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)
y_prob = lda.predict_proba(X_test)[:, 1]

In [ ]:
auc = roc_auc_score(y_test, y_prob)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)


In [ ]:
print(f"AUC: {auc:.4f}\nF1 Score: {f1:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}")

AUC: 0.9117
F1 Score: 0.8311
Precision: 0.8215
Recall: 0.8409
